# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（!マークに続けてOSコマンドを入力することで、任意のubuntu Linuxコマンドが入力可能）
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# require('RPostgreSQL')
require('tidyr')
require('dplyr')
require('stringr')
require('caret')
require('lubridate')
require('rsample')
require('unbalanced')

# host <- 'db'
# port <- Sys.getenv()["PG_PORT"]
# dbname <- Sys.getenv()["PG_DATABASE"]
# user <- Sys.getenv()["PG_USER"]
# password <- Sys.getenv()["PG_PASSWORD"]

# con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
# df_customer <- dbGetQuery(con,"SELECT * FROM customer")
# df_category <- dbGetQuery(con,"SELECT * FROM category")
# df_product <- dbGetQuery(con,"SELECT * FROM product")
# df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
# df_store <- dbGetQuery(con,"SELECT * FROM store")
# df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: tidyr
Warning message:
"package 'tidyr' was built under R version 3.6.3"Loading required package: dplyr
Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: stringr
Warning message:
"package 'stringr' was built under R version 3.6.3"Loading required package: caret
Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: rsample
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"

In [2]:
df_customer <- read.csv("./data/customer.csv", encoding="UTF-8")
head(df_customer, n = 5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS021313000114,"大野 あや子,1",女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS031415000172,"宇多田 貴美子,1",女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,
CS028811000001,"堀井 かおり,1",女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
# df_category <- read.csv("./data/category.csv", encoding="UTF-8")
df_category <- read.csv("./data/category.csv", encoding="shift-jis") # エラーがでたので文字コードを変更
head(df_category, n = 5)

category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
4,惣菜,401,御飯類,40101,弁当類
4,惣菜,401,御飯類,40102,寿司類
4,惣菜,402,佃煮類,40201,魚介佃煮類
4,惣菜,402,佃煮類,40202,海草佃煮類
4,惣菜,402,佃煮類,40203,野菜佃煮類


In [4]:
df_product <- read.csv("./data/product.csv", encoding="UTF-8")
head(df_product, n = 5)

X.U.FEFF.product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
P040101001,4,401,40101,198,149
P040101002,4,401,40101,218,164
P040101003,4,401,40101,230,173
P040101004,4,401,40101,248,186
P040101005,4,401,40101,268,201


In [5]:
df_receipt <- read.csv("./data/receipt.csv", encoding="UTF-8")
head(df_receipt, n = 5)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [6]:
df_store <- read.csv("./data/store.csv", encoding="shift-jis") # 正常に読まなかったので文字コードを変換した
head(df_store, n = 5)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679


In [7]:
df_geocode <- read.csv("./data/geocode.csv", encoding="UTF-8") # 時間がかかる, Python では一瞬で読めた
head(df_geocode, n = 5)

X.U.FEFF.postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
060-0000,"北海道,札幌市中央区",,,,北海道札幌市中央区,141.34103,43.05513,NA
064-0941,"北海道,札幌市中央区","旭ケ丘,",,"北海道札幌市中央区旭ケ丘,141.31972",43.04223,,NA,NA
060-0042,"北海道,札幌市中央区",大通西,,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102,NA
060-0042,"北海道,札幌市中央区",大通西,,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080,NA
060-0042,"北海道,札幌市中央区",大通西,,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086,NA


# 演習問題

---
> R-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [28]:
nrow(df_receipt)

[1] 104681

---
> R-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [11]:
unique(df_receipt$customer_id)

[1] CS006214000001 CS008415000097 CS028414000014 ZZ000000000000
   [5] CS025415000050 CS003515000195 CS024514000042 CS040415000178
   [9] CS027514000015 CS025415000134 CS021515000126 CS039414000052
  [13] CS016215000032 CS026515000042 CS034415000121 CS040414000073
  [17] CS026414000062 CS015414000118 CS034414000034 CS029415000133
  [21] CS007513000235 CS012515000143 CS011115000004 CS018415000198
  [25] CS018205000001 CS008515000124 CS023415000033 CS004415000232
  [29] CS037415000204 CS032515000109 CS010515000120 CS014415000077
  [33] CS030515000078 CS019412000185 CS020814000004 CS030512000087
  [37] CS005415000105 CS024514000012 CS026615000085 CS015415000120
  [41] CS027414000084 CS009415000022 CS028513000073 CS040415000290
  [45] CS015513000065 CS038614000007 CS019414000068 CS008314000069
  [49] CS028415000203 CS019514000062 CS005513000061 CS015415000061
  [53] CS014214000013 CS011415000277 CS002512000180 CS023415000240
  [57] CS019415000193 CS039415000129 CS004415000083 CS026414000043
  [61] CS022414000065 CS033415000244 CS041411000001 CS029414000020
  [65] CS001415000472 CS001515000532 CS047211000001 CS014414000032
  [69] CS021514000061 CS040514000003 CS004415000050 CS026513000189
  [73] CS023414000011 CS019215000046 CS023415000271 CS014411000026
  [77] CS004413000039 CS017415000158 CS030514000093 CS033414000053
  [81] CS016415000215 CS004515000083 CS018415000035 CS030415000197
  [85] CS003515000683 CS014415000021 CS004515000723 CS008415000018
  [89] CS013415000041 CS009415000038 CS033411000008 CS016515000182
  [93] CS009314000042 CS023214000011 CS003515000131 CS008414000013
  [97] CS032215000073 CS031415000086 CS034315000164 CS037214000004
 [101] CS029415000018 CS010513000187 CS015415000188 CS013514000061
 [105] CS006815000006 CS026515000200 CS003415000229 CS015515000176
 [109] CS033414000040 CS030214000008 CS010414000032 CS033515000125
 [113] CS002515000266 CS033515000083 CS021515000056 CS029415000224
 [117] CS008515000180 CS037415000079 CS013414000138 CS023515000195
 [121] CS039415000073 CS028513000074 CS006514000034 CS040515000174
 [125] CS002515000372 CS032214000014 CS021615000146 CS007615000066
 [129] CS018515000013 CS028415000049 CS017604000004 CS031414000059
 [133] CS010515000031 CS016415000069 CS007515000170 CS013415000188
 [137] CS032414000051 CS032414000091 CS037304000008 CS028315000123
 [141] CS027414000125 CS004415000169 CS005415000037 CS014515000039
 [145] CS021515000239 CS023515000126 CS003515000017 CS024512000067
 [149] CS034205000001 CS037513000136 CS020415000008 CS013415000214
 [153] CS035215000015 CS014314000052 CS008414000053 CS031814000006
 [157] CS026613000065 CS019512000123 CS002215000038 CS007512000139
 [161] CS033515000070 CS025414000112 CS028414000005 CS040415000121
 [165] CS014513000109 CS031515000016 CS035513000195 CS013415000002
 [169] CS039514000101 CS030415000264 CS024214000020 CS034214000029
 [173] CS002615000079 CS001414000056 CS014215000047 CS015515000077
 [177] CS011514000050 CS024415000131 CS012415000043 CS016411000003
 [181] CS011415000157 CS033515000112 CS025415000149 CS029415000194
 [185] CS029515000141 CS015513000041 CS023304000005 CS028515000032
 [189] CS009414000082 CS032115000007 CS009515000110 CS029415000209
 [193] CS019414000049 CS035515000170 CS028215000022 CS046415000011
 [197] CS019414000012 CS009515000084 CS013415000053 CS019415000005
 [201] CS027415000045 CS011415000057 CS003515000058 CS018414000065
 [205] CS021515000089 CS035614000003 CS024304000003 CS032415000002
 [209] CS023513000104 CS023515000234 CS029305000006 CS037515000126
 [213] CS018415000039 CS023515000167 CS006515000175 CS035515000021
 [217] CS007414000032 CS030414000048 CS029514000026 CS011215000001
 [221] CS017515000197 CS021512000095 CS014414000063 CS030414000059
 [225] CS035415000289 CS022512000070 CS015514000095 CS015515000239
 [229] CS006415000221 CS017515000095 CS002414000326 CS025414000016
 [233] CS031414000048 CS029215000025 CS023514000086 CS009205000002
 [237] CS031415000231 CS004215000154 CS01821500005

In [12]:
length(unique(df_receipt$customer_id))

[1] 8307

---
> R-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [30]:
df_receipt %>%
    group_by(store_cd) %>%
    summarise(amount=sum(amount),
              quantity=sum(quantity))

`summarise()` ungrouping output (override with `.groups` argument)


store_cd,amount,quantity
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403
S13001,811936,2347
S13002,727821,2340
S13003,764294,2197
S13004,779373,2390
S13005,629876,2004


In [13]:
# 1つずつ見ていきましょう --> これに summarize() 関数をハイプで作用させた
df_receipt %>% group_by(store_cd)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138
20181205,1259971200,S14024,1102,2,CS024514000042,P080101005,1,30
20190922,1285113600,S14040,1102,1,CS040415000178,P070501004,1,128
20170504,1209859200,S13020,1112,2,ZZ000000000000,P071302010,1,770
20191010,1286668800,S14027,1102,1,CS027514000015,P071101003,1,680


---
> R-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [14]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(max_ymd=max(sales_ymd)) %>%
    slice(1:10)

`summarise()` ungrouping output (override with `.groups` argument)


customer_id,max_ymd
CS001113000004,20190308
CS001114000005,20190731
CS001115000010,20190405
CS001205000004,20190625
CS001205000006,20190224
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811
CS001212000070,20191018


パイプ、チョー便利ですね。

---
> R-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [32]:
df_receipt %>%
    group_by(customer_id) %>%
    summarise(min_ymd=min(sales_ymd)) %>%
    slice(1:10)

`summarise()` ungrouping output (override with `.groups` argument)


customer_id,min_ymd
CS001113000004,20190308
CS001114000005,20180503
CS001115000010,20171228
CS001205000004,20170914
CS001205000006,20180207
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811
CS001212000070,20191018
